In [ ]:
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import accuracy_score

import random

In [ ]:
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# data processing

In [ ]:
# same as sklearn 
df = pd.read_csv("data/allHorizonData_cut.csv")

seq_features = ['r1', 'r2', 'r3', 'r4', 'c1', 'c2', 'c3', 'c4', 'rt1', 'rt2', 'rt3', 'rt4']
static_features = ['gameLength', 'uc']

target = 'c5'

for subj_id, subj_df in df.groupby('subjectID'):
    X_seq = df[seq_features]
    X_static = df[static_features]
    y = df[target]
    y = y-1 # binary CE only takes in 0, 1

    

